
# An Introduction to Model Definition and Simulation using Epydemix

Welcome to this tutorial on **epydemix**, a Python library designed for epidemiological modeling and simulations. In this tutorial, we will guide you through the process of defining a simple epidemiological model and running simulations to explore how diseases spread over time.

We will start by explaining the key concepts in epidemiological modeling, and then move on to hands-on examples using **epydemix**.

Let’s begin by setting up our environment!



## Installation

First, if you haven’t already installed **epydemix**, you can do so by running the following command:


In [ ]:
#!pip install epydemix


Now, let’s import the necessary modules.


In [1]:
# Import the necessary components
from epydemix import EpiModel
from epydemix.visualization import plot_quantiles

## Understanding the Basics of Epidemiological Modeling

Before diving into code, let’s quickly go over the basic structure of an epidemiological compartmental model.

Compartmental models are used to describe how diseases spread through populations. These models divide the population into compartments, each representing a specific state in the progression of the disease. One of the simplest models is the **SIR model**, where:

- **S** represents the susceptible individuals (those who can get infected).
- **I** represents the infected individuals (those who are currently carrying the disease).
- **R** represents the recovered individuals (those who have recovered and are immune).

In compartmental models, individuals move between compartments through transitions, which are of two main types:

- **Mediated Transitions**: These transitions occur due to interactions between individuals in different compartments. In the case of the SIR model, the transition from **S** (susceptible) to **I** (infected) is mediated by the presence of infected individuals. The rate at which susceptibles become infected depends on both the number of infected individuals and the transmission rate.
  
  For example, the transition from **S** to **I** happens at a rate proportional to:
  ```
  Transition rate = β × S × I
  ```
  where:
  - **β** is the transmission rate,
  - **S** is the number of susceptible individuals,
  - **I** is the number of infected individuals.

- **Spontaneous Transitions**: These transitions happen independently of interactions with other compartments. For instance, in the SIR model, the transition from **I** (infected) to **R** (recovered) is spontaneous because infected individuals recover based on a fixed recovery rate without needing to interact with any other compartment.
  
  This transition from **I** to **R** occurs at a rate proportional to:
  ```
  Transition rate = γ × I
  ```
  where:
  - **γ** is the recovery rate,
  - **I** is the number of infected individuals.

The dynamics of the disease over time are shaped by these transitions, as individuals move between susceptible, infected, and recovered states.

Now, let's see how we can define a simple SIR model using **epydemix**.



## Defining a Basic Model

In **epydemix**, defining a model is as simple as specifying the compartments and parameters that govern the dynamics of disease transmission. Let’s start with a **Basic SIR Model**:


In [2]:
# Defining a basic SIR model
sir_model = EpiModel(
    name='SIR Model',
    compartments=['S', 'I', 'R'],  # Susceptible, Infected, Recovered
    parameters={
        'beta': 0.3,  # Transmission rate
        'gamma': 0.1  # Recovery rate
    }
)


TypeError: __init__() got an unexpected keyword argument 'parameters'


## Initializing a Simulation

Now that we’ve defined our model, it’s time to set up a simulation. A simulation in **epydemix** requires:
- **Initial conditions**: This describes how many individuals start in each compartment at the beginning of the simulation.
- **Time steps**: The duration for which we want to simulate the epidemic.

Let's set up a simulation where 990 people start off susceptible, 10 people are infected, and no one has recovered yet.


In [ ]:
# Initial conditions
initial_conditions = {
    'S': 990,  # 990 people are susceptible
    'I': 10,   # 10 people are initially infected
    'R': 0     # 0 people have recovered at the start
}

# Creating the simulation
sir_simulation = Simulation(
    model=sir_model,
    initial_conditions=initial_conditions,
    time_steps=100  # We will simulate the epidemic over 100 time steps
)



## Running the Simulation

With the model and simulation initialized, we can now run the simulation and observe the results. The results will show how the number of susceptible, infected, and recovered individuals changes over time.


In [ ]:
# Running the simulation
results = sir_simulation.run()

# Let's take a quick look at the results
print(results.head())  # Display the first few rows of the results



## Visualizing the Results

**Epydemix** comes with built-in visualization tools that make it easy to see how the simulation progresses. Let’s create a plot to visualize the evolution of the compartments over time.


In [ ]:
# Visualizing the simulation results
plotting.plot_simulation_results(results)



## Exploring Model Customizations

Now that we have a basic model running, we can start customizing it to explore different scenarios. For example, we could extend the model to include additional compartments, such as vaccinated individuals, or adjust the transmission and recovery rates.

Let’s see how we could add a **vaccination compartment** to our model.


In [ ]:
# Extending the model to include vaccination
sirv_model = Model(
    name='SIRV Model',
    compartments=['S', 'I', 'R', 'V'],  # Adding vaccinated compartment
    parameters={
        'beta': 0.3,
        'gamma': 0.1,
        'vaccination_rate': 0.05  # New vaccination rate parameter
    }
)



## Conclusion

Congratulations! You have successfully defined and simulated an epidemiological model using **epydemix**. You now know how to:
- Define a simple epidemiological model (SIR).
- Set up initial conditions and run a simulation.
- Visualize the results and interpret the disease dynamics.
- Extend the model to include additional compartments like vaccination.

From here, you can experiment with different model configurations and simulations, such as adjusting the transmission rate, adding more compartments, or testing different scenarios.

## Further Exploration
- Experiment with different values for `beta` and `gamma` to see how they affect the dynamics.
- Try adding a birth and death rate to your model.
- Explore how different vaccination rates impact the spread of the disease.
